https://github.com/alirezadir/Machine-Learning-Interviews/blob/main/src/MLC/notebooks/logistic_regression.ipynb

1. 선형결합 후 sigmoid하면 되는거 아녀..?
- 로지스틱 회귀에서 사용하는 비용 함수는 로그 손실(Log Loss) 또는 이진 교차 엔트로피(Binary Cross-Entropy)
- cost = (-1 / n_samples) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
- dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
- db = (1 / n_samples) * np.sum(y_pred - y)
- MSE아니다~~~ 회귀 아니다~~~

2. 경사하강법
- 모든 데이터 포인트에 대해 n_iter회 반복
- 각 데이터 포인트에 대해 cost = loss + regulation 미분 값에 값 대입하여 gradients 구하기
- 가중치에 -lr * gradients 누적

3. mini-batch SGD ... optimizer 적용
- n_samples: 전체 X 통째로 계산(batch) 
- batch_size: batch_size 만큼 X 랜덤 샘플링해서 계산

In [ ]:
import numpy as np

class LogisticRegression:
    def __init__(self, lr=0.01, regl=0.01):
        self.W=None
        self.lr=lr
        self.regl=regl
        
    def sigmoid(self, z):
        return 1/(1+(np.exp((-z))))
        
    def predict(self, x):
        z= np.dot(self.W, x) 
        pred_y= sigmoid(z)
#         return np.round(y_pred).astype(int)
        return pred_y
    
    def fit(self, X, Y, n_iter=10):
        if X is None or len(X) != len(Y):
            raise ValueError("Sample length is not valid")
        
#         initialize weights and bias to zeros  샘플수가 아니라 피쳐수 입니다(^^
#         n_samples, n_features = X.shape
#         self.weights = np.zeros(n_features)
#         self.bias = 0
        n =len(X)
        self.W=[np.ones(n,1), np.zeros(n,n)]
        
        for epoch in range(n_iter):
            for i in range(n):
                x,y=X[i],Y[i]
                z= np.dot(self.W, x)
                pred_y=sigmoid(z)
                # np.sign, np.log, np.sum
                grad= 2*(y-pred_y)*(np.dot(x, sigmoid(z)))+2*self.regl*abs(sum(self.W[i]))
                self.W -= self.lr * self.W
    

In [ ]:
import numpy as np

class LogisticRegression:
    
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        
    def fit(self, X, y):
        # initialize weights and bias to zeros
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # gradient descent optimization
        for i in range(self.n_iters):
            # calculate predicted probabilities and cost
            z = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(z)
            #cost = (-1 / n_samples) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
            
            # calculate gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)
            
            # update weights and bias
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
    def predict(self, X):
        # calculate predicted probabilities
        z = np.dot(X, self.weights) + self.bias
        y_pred = self._sigmoid(z)
        # convert probabilities to binary predictions
        return np.round(y_pred).astype(int)
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

In [ ]:
import numpy as np

class LogisticRegression:
    
    def __init__(self, learning_rate=0.01, n_iters=1000, regularization='l2', reg_strength=0.1, batch_size=32):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.regularization = regularization
        self.reg_strength = reg_strength
        self.batch_size = batch_size
        self.weights = None
        self.bias = None
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        n_batches = n_samples // self.batch_size
        for i in range(self.n_iters):
            batch_indices = np.random.choice(n_samples, self.batch_size)
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            z = np.dot(X_batch, self.weights) + self.bias
            y_pred = self._sigmoid(z)
#             cost = (-1 / self.batch_size) * np.sum(y_batch * np.log(y_pred) + (1 - y_batch) * np.log(1 - y_pred))
#             if self.regularization == 'l2':
#                 reg_cost = (self.reg_strength / (2 * n_samples)) * np.sum(self.weights ** 2)
#                 cost += reg_cost
#             elif self.regularization == 'l1':
#                 reg_cost = (self.reg_strength / (2 * n_samples)) * np.sum(np.abs(self.weights))
#                 cost += reg_cost
            dw = (1 / self.batch_size) * np.dot(X_batch.T, (y_pred - y_batch))
            db = (1 / self.batch_size) * np.sum(y_pred - y_batch)
            if self.regularization == 'l2':
                dw += (self.reg_strength / n_samples) * self.weights
            elif self.regularization == 'l1':
                dw += (self.reg_strength / n_samples) * np.sign(self.weights)
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        y_pred = self._sigmoid(z)
        return np.round(y_pred).astype(int)
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))